<a href="https://colab.research.google.com/github/keerthnn/AIML/blob/main/Astar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.A Star**

In [ ]:
def astarAlgo(start_node, stop_node):
    open_set = set([start_node])
    closed_set = set()
    g = {}
    parents = {}
    g[start_node] = 0
    parents[start_node] = start_node

    while len(open_set) > 0:
        n = None
        for v in open_set:
            if n is None or g[v] + heuristic(v) < g[n] + heuristic(n):
                n = v

        if n == stop_node or graph_nodes[n] is None:
            pass
        else:
            for (m, weight) in get_neighbours(n):
                if m not in open_set and m not in closed_set:
                    open_set.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n

                    if m in closed_set:
                        closed_set.remove(m)
                        open_set.add(m)

        if n is None:
            print('Path Doesn\'t Exist!')
            return None

        if n == stop_node:
            path = []
            while parents[n] != n:
                path.append(n)
                n = parents[n]
            path.append(start_node)
            path.reverse()
            print('Path Found:', format(path))
            return path

        open_set.remove(n)
        closed_set.add(n)

    print('Path Doesn\'t Exist!')
    return None


def get_neighbours(v):
    if v in graph_nodes:
        return graph_nodes[v]
    else:
        return None


def heuristic(n):
    H_dist = {
        'A': 10,
        'B': 8,
        'C': 5,
        'D': 7,
        'E': 3,
        'F': 6,
        'G': 5,
        'H': 4,
        'I': 1,
        'J': 0
    }
    return H_dist[n]


graph_nodes = {
    'A': [('B', 6), ('F', 3)],
    'B': [('C', 3), ('D', 2)],
    'C': [('D', 1), ('E', 5)],
    'D': [('C', 1), ('E', 8)],
    'E': [('I', 5), ('J', 5)],
    'F': [('G', 1), ('H', 7)],
    'G': [('I', 3)],
    'H': [('I', 2)],
    'I': [('E', 5), ('J', 3)]
}

astarAlgo('A', 'J')


Path Found: ['A', 'F', 'G', 'I', 'J']


['A', 'F', 'G', 'I', 'J']

# **2.AO STAR**

In [ ]:
def recAOStar(n):
    global finalPath
    print("Expanding Node :", n)
    and_nodes = []
    or_nodes = []
    if n in allNodes:
        if 'AND' in allNodes[n]:
            and_nodes = allNodes[n]['AND']
        if 'OR' in allNodes[n]:
            or_nodes = allNodes[n]['OR']
    if len(and_nodes) == 0 and len(or_nodes) == 0:
        return
    solvable = False
    marked = {}
    while not solvable:
        if len(marked) == len(and_nodes) + len(or_nodes):
            min_cost_least, min_cost_group_least = least_cost_group(and_nodes, or_nodes, {})
            solvable = True
            change_heuristic(n, min_cost_least)
            optimal_child_group[n] = min_cost_group_least
            continue
        min_cost, min_cost_group = least_cost_group(and_nodes, or_nodes, marked)
        is_expanded = False

        if len(min_cost_group) > 1:
            if min_cost_group[0] in allNodes:
                is_expanded = True
                recAOStar(min_cost_group[0])
            if min_cost_group[1] in allNodes:
                is_expanded = True
                recAOStar(min_cost_group[1])
        else:
            if min_cost_group in allNodes:
                is_expanded = True
                recAOStar(min_cost_group)
        if is_expanded:
            min_cost_verify, min_cost_group_verify = least_cost_group(and_nodes, or_nodes, {})
            if min_cost_group == min_cost_group_verify:
                solvable = True
                change_heuristic(n, min_cost_verify)
                optimal_child_group[n] = min_cost_group
        else:
            solvable = True
            change_heuristic(n, min_cost)
            optimal_child_group[n] = min_cost_group
        marked[min_cost_group] = 1
    return heuristic(n)


def least_cost_group(and_nodes, or_nodes, marked):
    node_wise_cost = {}
    for node_pair in and_nodes:
        if not node_pair[0] + node_pair[1] in marked:
            cost = 0
            cost = cost + heuristic(node_pair[0]) + heuristic(node_pair[1]) + 2
            node_wise_cost[node_pair[0] + node_pair[1]] = cost
    for node in or_nodes:
        if not node in marked:
            cost = 0
            cost = cost + heuristic(node) + 1
            node_wise_cost[node] = cost
    min_cost = 999999
    min_cost_group = None
    for costKey in node_wise_cost:
        if node_wise_cost[costKey] < min_cost:
            min_cost = node_wise_cost[costKey]
            min_cost_group = costKey
    return [min_cost, min_cost_group]


def heuristic(n):
    return H_dist[n]


def change_heuristic(n, cost):
    H_dist[n] = cost
    return


def print_path(node):
    print(optimal_child_group[node], end="")
    node = optimal_child_group[node]
    if len(node) > 1:
        if node[0] in optimal_child_group:
            print("->", end="")
            print_path(node[0])
        if node[1] in optimal_child_group:
            print("->", end="")
            print_path(node[1])
    else:
        if node in optimal_child_group:
            print("->", end="")
            print_path(node)


H_dist = {
    'A': -1,
    'B': 4,
    'C': 2,
    'D': 3,
    'E': 6,
    'F': 8,
    'G': 2,
    'H': 0,
    'I': 0,
    'J': 0
}

allNodes = {
    'A': {'AND': [('C', 'D')], 'OR': ['B']},
    'B': {'OR': ['E', 'F']},
    'C': {'AND': [('H', 'I')], 'OR': ['G']},
    'D': {'OR': ['J']}
}

optimal_child_group = {}
optimal_cost = recAOStar('A')
print('Nodes Which Give Optimal Cost Are')
print_path('A')
print('\nOptimal Cost Is:', optimal_cost)


Expanding Node : A
Expanding Node : B
Expanding Node : C
Expanding Node : D
Nodes Which Give Optimal Cost Are
CD->HI->J
Optimal Cost Is: 5


In [12]:
import pandas as pd
import math

def infoGain(P, N):
    return -P / (P + N) * math.log2(P / (P + N)) - N / (P + N) * math.log2(N / (P + N))

def insertNode(tree, addTo, Node):
    for k, v in tree.items():
        if isinstance(v, dict):
            tree[k] = insertNode(v, addTo, Node)
        if addTo in tree:
            tree[addTo] = {Node: 'None'} if isinstance(tree[addTo], dict) else {Node: 'None'}
    return tree

def insertConcept(tree, addTo, Node):
    for k, v in tree.items():
        if isinstance(v, dict):
            tree[k] = insertConcept(v, addTo, Node)
        if addTo in tree:
            tree[addTo] = Node
    return tree

def getNextNode(data, AttributeList, concept, conceptVals, tree, addTo):
    Total = data.shape[0]
    if Total == 0:
        return tree

    countC = {cVal: data[data[concept] == cVal].shape[0] for cVal in conceptVals}

    if countC[conceptVals[0]] == 0:
        tree = insertConcept(tree, addTo, conceptVals[1])
        return tree

    if countC[conceptVals[1]] == 0:
        tree = insertConcept(tree, addTo, conceptVals[0])
        return tree

    ClassEntropy = infoGain(countC[conceptVals[0]], countC[conceptVals[1]])

    Attr = {a: list(set(data[a])) for a in AttributeList}
    AttrCount = {}
    EntropyAttr = {}

    for att in Attr:
        for vals in Attr[att]:
            for c in conceptVals:
                iData = data[data[att] == vals]
                dataAtt = iData[iData[concept] == c]
                AttrCount[c] = dataAtt.shape[0]

    TotalInfo = AttrCount[conceptVals[0]] + AttrCount[conceptVals[1]]

    if AttrCount[conceptVals[0]] == 0 or AttrCount[conceptVals[1]] == 0:
        InfoGain = 0
    else:
        InfoGain = infoGain(AttrCount[conceptVals[0]], AttrCount[conceptVals[1]])

    for att in Attr:
        EntropyAttr[att] = (TotalInfo / Total) * InfoGain if att not in EntropyAttr else EntropyAttr[att] + (TotalInfo / Total) * InfoGain

    Gain = {g: ClassEntropy - EntropyAttr[g] for g in EntropyAttr}
    Node = max(Gain, key=Gain.get)

    tree = insertNode(tree, addTo, Node)

    for nD in Attr[Node]:
        tree = insertNode(tree, Node, nD)
        newData = data[data[Node] == nD].drop(Node, axis=1)
        AttributeList = list(newData)[:-1]
        tree = getNextNode(newData, AttributeList, concept, conceptVals, tree, nD)

    return tree

def main():
    data = pd.read_csv('PlayTennis.csv')
    AttributeList = list(data)[:-1]
    concept = str(list(data)[-1])
    conceptVals = list(set(data[concept]))
    tree = getNextNode(data, AttributeList, concept, conceptVals, {'root': 'None'}, 'root')
    return tree

tree = main()['root']
df = pd.read_csv('PlayTennis.csv')

def test(tree, d):
    for k in tree:
        for v in tree[k]:
            if (d[k] == v and isinstance(tree[k][v], dict)):
                test(tree[k][v], d)
            elif (d[k] == v):
                print("Classification: " + tree[k][v])

df.head()
print(tree)
test(tree, df.loc[0, :])


{'Outlook': {'Sunny': {'Temperature': {'Cool': 'Yes'}}}}


# **3.Decision Tree**

In [13]:
import pandas as pd
import math

def infoGain(P, N):
    return -P / (P + N) * math.log2(P / (P + N)) - N / (P + N) * math.log2(N / (P + N))

def insertNode(tree, addTo, Node):
    for k, v in tree.items():
        if isinstance(v, dict):
            tree[k] = insertNode(v, addTo, Node)
        if addTo in tree:
            if isinstance(tree[addTo], dict):
                tree[addTo][Node] = 'None'
            else:
                tree[addTo] = {Node: 'None'}
    return tree

def insertConcept(tree, addTo, Node):
    for k, v in tree.items():
        if isinstance(v, dict):
            tree[k] = insertConcept(v, addTo, Node)
        if addTo in tree:
            tree[addTo] = Node
    return tree

def getNextNode(data, AttributeList, concept, conceptVals, tree, addTo):
    Total = data.shape[0]
    if Total == 0:
        return tree

    countC = {}
    for cVal in conceptVals:
        dataCC = data[data[concept] == cVal]
        countC[cVal] = dataCC.shape[0]

    if countC[conceptVals[0]] == 0:
        tree = insertConcept(tree, addTo, conceptVals[1])
        return tree

    if countC[conceptVals[1]] == 0:
        tree = insertConcept(tree, addTo, conceptVals[0])
        return tree

    ClassEntropy = infoGain(countC[conceptVals[0]], countC[conceptVals[1]])

    Attr = {}
    for a in AttributeList:
        Attr[a] = list(set(data[a]))

    AttrCount = {}
    EntropyAttr = {}

    for att in Attr:
        for vals in Attr[att]:
            for c in conceptVals:
                iData = data[data[att] == vals]
                dataAtt = iData[iData[concept] == c]
                AttrCount[c] = dataAtt.shape[0]

    TotalInfo = AttrCount[conceptVals[0]] + AttrCount[conceptVals[1]]

    if AttrCount[conceptVals[0]] == 0 or AttrCount[conceptVals[1]] == 0:
        InfoGain = 0
    else:
        InfoGain = infoGain(AttrCount[conceptVals[0]], AttrCount[conceptVals[1]])

    for att in Attr:
        if att not in EntropyAttr:
            EntropyAttr[att] = (TotalInfo / Total) * InfoGain
        else:
            EntropyAttr[att] = EntropyAttr[att] + (TotalInfo / Total) * InfoGain

    Gain = {}
    for g in EntropyAttr:
        Gain[g] = ClassEntropy - EntropyAttr[g]
    Node = max(Gain, key=Gain.get)

    tree = insertNode(tree, addTo, Node)

    for nD in Attr[Node]:
        tree = insertNode(tree, Node, nD)
        newData = data[data[Node] == nD].drop(Node, axis=1)
        AttributeList = list(newData)[:-1]
        tree = getNextNode(newData, AttributeList, concept, conceptVals, tree, nD)

    return tree

def main():
    data = pd.read_csv('PlayTennis.csv')

    # if 'Unnamed: 0' in data.columns:
    #     data = data.drop('Unnamed: 0', axis=1)

    AttributeList = list(data)[:-1]
    concept = str(list(data)[-1])
    conceptVals = list(set(data[concept]))

    tree = getNextNode(data, AttributeList, concept, conceptVals, {'root': 'None'}, 'root')

    return tree

tree = main()['root']
df = pd.read_csv('PlayTennis.csv')

def test(tree, d):
    for k in tree:
        for v in tree[k]:
            if (d[k] == v and isinstance(tree[k][v], dict)):
                test(tree[k][v], d)
            elif (d[k] == v):
                print("Classification: " + tree[k][v])

# if 'Unnamed: 0' in df.columns:
#     df = df.drop('Unnamed: 0', axis=1)

df.head()
print(tree)
test(tree, df.loc[0, :])


{'Outlook': {'Rain': {'Temperature': {'Mild': {'Humidity': {'Normal': 'Yes', 'High': 'No'}}, 'Cool': 'Yes', 'Hot': 'No'}}, 'Overcast': 'Yes', 'Sunny': {'Temperature': {'Mild': {'Humidity': {'Normal': 'Yes', 'High': 'No'}}, 'Hot': 'No', 'Cool': 'Yes'}}}}
Classification: No


# **5.NaïveBayesianClassifier**

In [10]:
import pandas as pd

def probAttr(data, attr, val):
    Total = data.shape[0]
    cnt = len(data[data[attr] == val])
    return cnt, cnt / Total

def train(data, Attr, conceptVals, concept):
    conceptProbs = {}
    countConcept = {}

    for cVal in conceptVals:
        countConcept[cVal], conceptProbs[cVal] = probAttr(data, concept, cVal)

    AttrConcept = {}
    probability_list = {}

    for att in Attr:
        probability_list[att] = {}
        AttrConcept[att] = {}

        for val in Attr[att]:
            AttrConcept[att][val] = {}
            a, probability_list[att][val] = probAttr(data, att, val)

            for cVal in conceptVals:
                dataTemp = data[data[att] == val]
                AttrConcept[att][val][cVal] = len(dataTemp[dataTemp[concept] == cVal]) / countConcept[cVal]

    print("P(A) : ", conceptProbs,"\n")
    print("P(X/A) : ", AttrConcept,"\n")
    print("P(X) : ", probability_list,"\n")

    return conceptProbs, AttrConcept, probability_list

def test(examples, Attr, concept_list, conceptProbs, AttrConcept, probability_list):
    misclassification_count = 0
    Total = len(examples)

    for ex in examples:
        px = {}

        for a in Attr:
            for x in ex:
                for c in concept_list:
                    if x in AttrConcept[a]:
                        if c not in px:
                            px[c] = conceptProbs[c] * AttrConcept[a][x][c] / probability_list[a][x]
                        else:
                            px[c] = px[c] * AttrConcept[a][x][c] / probability_list[a][x]

        print(px)
        classification = max(px, key=px.get)
        print("Classification :", classification, "Expected :", ex[-1])

        if classification != ex[-1]:
            misclassification_count += 1

    misclassification_rate = misclassification_count * 100 / Total
    accuracy = 100 - misclassification_rate

    print("Misclassification Count = {}".format(misclassification_count))
    print("Misclassification Rate = {}%".format(misclassification_rate))
    print("Accuracy = {}%".format(accuracy))

data = pd.read_csv('PlayTennis.csv')
#data.drop(['Unnamed: 0'], axis=1, inplace=True)
print(data)

concept = str(list(data)[-1])
print(concept)

concept_list = set(data[concept])
print(concept_list)

Attr = {}
for a in list(data)[:-1]:
    Attr[a] = set(data[a])
print(Attr)

conceptProbs, AttrConcept, probability_list = train(data, Attr, concept_list, concept)
examples = pd.read_csv('PlayTennis.csv')
test(examples.values, Attr, concept_list, conceptProbs, AttrConcept, probability_list)


     Outlook Temperature Humidity    Wind Play Tennis
0      Sunny         Hot     High    Weak          No
1      Sunny         Hot     High  Strong          No
2   Overcast         Hot     High    Weak         Yes
3       Rain        Mild     High    Weak         Yes
4       Rain        Cool   Normal    Weak         Yes
5       Rain        Cool   Normal  Strong          No
6   Overcast        Cool   Normal  Strong         Yes
7      Sunny        Mild     High    Weak          No
8      Sunny        Cool   Normal    Weak         Yes
9       Rain        Mild   Normal    Weak         Yes
10     Sunny        Mild   Normal  Strong         Yes
11  Overcast        Mild     High  Strong         Yes
12  Overcast         Hot   Normal    Weak         Yes
13      Rain        Mild     High  Strong          No
Play Tennis
{'No', 'Yes'}
{'Outlook': {'Rain', 'Overcast', 'Sunny'}, 'Temperature': {'Mild', 'Hot', 'Cool'}, 'Humidity': {'Normal', 'High'}, 'Wind': {'Weak', 'Strong'}}
P(A) :  {'No': 0.3571

# **Candidate Elimanation**

In [ ]:
import pandas as pd

df = pd.read_csv('Datasets/EnjoySports.csv')
# df = df.drop(['slno'], axis=1)
concepts = df.values[:, :-1]
target = df.values[:, -1]
df.head()

def learn(concepts, target):
    specific_h = concepts[0].copy()
    general_h = [["?" for i in range(len(specific_h))] for i in range(len(specific_h))]

    for i, h in enumerate(concepts):
        if target[i] == "yes":
            for x in range(len(specific_h)):
                if h[x] != specific_h[x]:
                    specific_h[x] = '?'
                    general_h[x][x] = '?'

        if target[i] == "no":
            for x in range(len(specific_h)):
                if h[x] != specific_h[x]:
                    general_h[x][x] = specific_h[x]
                else:
                    general_h[x][x] = '?'

    indices = [i for i, val in enumerate(general_h) if val == ['?'] * len(specific_h)]
    for i in indices:
        general_h.remove(['?'] * len(specific_h))

    return specific_h, general_h

s_final, g_final = learn(concepts, target)
print(f"Final S : {s_final}")
print(f"Final G : {g_final}")


# **6.ANN**

In [ ]:
import numpy as np

X = np.array(([2, 9], [1, 5], [3, 6]), dtype=float)
y = np.array(([92], [86], [89]), dtype=float)
X = X / np.amax(X, axis=0)
y = y / 100


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def derivatives_sigmoid(x):
    return x * (1 - x)

epoch = 1000
learning_rate = 0.6
inputlayer_neurons = 2
hiddenlayer_neurons = 3
output_neurons = 1

wh = np.random.uniform(size=(inputlayer_neurons, hiddenlayer_neurons))
bh = np.random.uniform(size=(1, hiddenlayer_neurons))
wo = np.random.uniform(size=(hiddenlayer_neurons, output_neurons))
bo = np.random.uniform(size=(1, output_neurons))

for i in range(epoch):
    net_h = np.dot(X, wh) + bh
    sigma_h = sigmoid(net_h)
    net_o = np.dot(sigma_h, wo) + bo
    output = sigmoid(net_o)


    deltaK = (y - output) * derivatives_sigmoid(output)
    deltaH = deltaK.dot(wo.T) * derivatives_sigmoid(sigma_h)
    wo = wo + sigma_h.T.dot(deltaK) * learning_rate
    wh = wh + X.T.dot(deltaH) * learning_rate

print("Input: \n" + str(X))
print("Actual Output: \n" + str(y))
print("Predicted Output: \n", output)


# **7.KMeans**

In [ ]:
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

iris = datasets.load_iris()
X = pd.DataFrame(iris.data)
y = pd.DataFrame(iris.target)
X.columns = ['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width']
y.columns = ['Targets']

model = KMeans(n_clusters=3)
model.fit(X)

plt.figure(figsize=(14, 14))
colormap = np.array(['red', 'lime', 'black'])

plt.subplot(2, 2, 1)
plt.scatter(X.Petal_Length, X.Petal_Width, c=colormap[y.Targets], s=40)
plt.title('Real Clusters')
plt.xlabel('Petal Length')
plt.ylabel('Petal Width')

plt.subplot(2, 2, 2)
plt.scatter(X.Petal_Length, X.Petal_Width, c=colormap[model.labels_], s=40)
plt.title('K-Means Clustering')
plt.xlabel('Petal Length')
plt.ylabel('Petal Width')

from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaler.fit(X)
xsa = scaler.transform(X)
xs = pd.DataFrame(xsa, columns=X.columns)

from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=3)
gmm.fit(xs)
gmm_y = gmm.predict(xs)

plt.subplot(2, 2, 3)
plt.scatter(X.Petal_Length, X.Petal_Width, c=colormap[gmm_y], s=40)
plt.title('GMM Clustering')
plt.xlabel('Petal Length')
plt.ylabel('Petal Width')

print('Observation: The GMM Using EM Algorithm Based Clustering Matched The True Labels More Closely Than The K-Means.')


**# 8.KNN**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

dataset = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    dataset["data"], dataset["target"], random_state=0
)

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(X_train, y_train)

prediction = kn.predict(X_test)
conf_matrix = confusion_matrix(y_test, prediction)

print("Confusion Matrix:")
print(conf_matrix)


# **9.Locally Weighted Regressional**

In [ ]:
from math import ceil
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt

def lowess(x, y, f, iterations):
    n = len(x)
    r = int(ceil(f * n))
    h = [np.sort(np.abs(x - x[i]))[r] for i in range(n)]
    w = np.clip(np.abs((x[:, None] - x[None, :]) / h), 0.0, 1.0)
    w = (1 - w ** 3) ** 3
    yest = np.zeros(n)
    delta = np.ones(n)

    for iteration in range(iterations):
        for i in range(n):
            weights = delta * w[:, i]
            b = np.array([np.sum(weights * y), np.sum(weights * y * x)])
            A = np.array([[np.sum(weights), np.sum(weights * x)],
                          [np.sum(weights * x), np.sum(weights * x * x)]])
            beta = linalg.solve(A, b)
            yest[i] = beta[0] + beta[1] * x[i]

        residuals = y - yest
        s = np.median(np.abs(residuals))
        delta = np.clip(residuals / (6.0 * s), -1, 1)
        delta = (1 - delta ** 2) ** 2

    return yest

def main():
    n = 100
    x = np.linspace(0, 2 * np.pi, n)
    y = np.sin(x) + 0.3 * np.random.randn(n)
    f = 0.25
    iterations = 3
    yest = lowess(x, y, f, iterations)

    plt.plot(x, y, "r.", label="Original Data")
    plt.plot(x, yest, "b-", label="LOWESS Smoothing")
    plt.legend()
    plt.show()

if __name__ == "__main__":
    main()


In [8]:
import pandas as pd
from pprint import pprint
from sklearn.feature_selection import mutual_info_classif
from collections import Counter

def id3(df, target_attribute, attribute_names, default_class=None):
    cnt = Counter(x for x in df[target_attribute])
    if len(cnt) == 1:
        return next(iter(cnt))

    elif df.empty or (not attribute_names):
        return default_class

    else:
        gainz = mutual_info_classif(df[attribute_names], df[target_attribute], discrete_features=True)
        index_of_max = gainz.tolist().index(max(gainz))
        best_attr = attribute_names[index_of_max]
        tree = {best_attr: {}}
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]

        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset, target_attribute, remaining_attribute_names, default_class)
            tree[best_attr][attr_val] = subtree

        return tree

df = pd.read_csv("PlayTennis.csv")

# Extracting attribute names excluding the target attribute
attribute_names = df.columns.tolist()
attribute_names.remove("Play Tennis")

for colname in df.select_dtypes("object"):
    df[colname], _ = df[colname].factorize()

print(df)

tree = id3(df, "Play Tennis", attribute_names)
print("The tree structure")
pprint(tree)


    Outlook  Temperature  Humidity  Wind  Play Tennis
0         0            0         0     0            0
1         0            0         0     1            0
2         1            0         0     0            1
3         2            1         0     0            1
4         2            2         1     0            1
5         2            2         1     1            0
6         1            2         1     1            1
7         0            1         0     0            0
8         0            2         1     0            1
9         2            1         1     0            1
10        0            1         1     1            1
11        1            1         0     1            1
12        1            0         1     0            1
13        2            1         0     1            0
The tree structure
{'Outlook': {0: {'Humidity': {0: 0, 1: 1}}, 1: 1, 2: {'Wind': {0: 1, 1: 0}}}}
